about pLSA : https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/05/25/plsa/

## LDA

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import re

petition = pd.read_csv('petition.csv', parse_dates=['start','end'], index_col=0)
petition.head(3)

,start,end,answered,votes,category,title,content
article_id,,,,,,,
21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...
23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."


In [2]:
petition['category'].unique()

array(['안전/환경', '기타', '미래', '일자리', '보건복지', '정치개혁', '경제민주화', '인권/성평등',
       '외교/통일/국방', '육아/교육', '문화/예술/체육/언론', '반려동물', '교통/건축/국토', '행정',
       '농산어촌', '저출산/고령화대책', '성장동력'], dtype=object)

### '안전/환경','보건복지' 두 개의 주제를 Topic Modeling 해보겠습니다.

In [4]:
data = petition.loc[petition['category'].isin(['안전/환경','보건복지'])]

In [6]:
data.shape

(55486, 7)

전처리를 진행합니다.

In [7]:
def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    # 공백이 여러개 있는 것을 하나로 줄여준다.
    text = re.sub(' +', ' ', text)
    return text

In [124]:
def remove_stopwords(text):
    stops = ['수', '있는', '있습니다','습니다', '그', '년도', '에', '합니다', 
             '하는', '및', '제', '할', '하고', '더', '대한', '한', 
             '그리고', '월', '저는', '없는', '것입니다', '등', '일', 
             '많은', '이런', '것은', '왜', '같은', 
             '없습니다', '위해', '한다' ,'이', '드립니다',
            '부탁드립니다','아닌가요','그런가요','바랍니다','아닙니다','감사합니다',
            '아닌가', '아니고', '바랍니다']
    meaningful_words = [w for w in text if not w in stops]
    return ' '.join(meaningful_words)

In [125]:
from konlpy.tag import Mecab

tokenizer = Mecab()

In [126]:
# 한글, 영문만 남기고 모두 제거
data['content_preprocessing'] = data['content'].apply(preprocessing)
# 토큰화
data['content_preprocessing'] = data['content_preprocessing'].apply(tokenizer.morphs)
# stopwords
data['content_preprocessing'] = data['content_preprocessing'].apply(remove_stopwords)

In [128]:
data['content_preprocessing'][:5]

article_id
21    스텔라 데 이지호 제안 입니다 스텔라 데 이지호 가 침몰 하 고 달째 가 되 고 있...
26    청원 인 왕 기 운 서울 도봉구 창동 삼성 아파트 호 청원 취지 보건 복지 부 부령...
31    안녕 하 세요 대통령 님 금연 관련 정책 들 강화 되 고 있 지만 여전히 흡연자 의...
53    안녕 하 세요 저 는 부산 예문 여고 학년 재학 중 인 학생 입니다 다름 아니 라 ...
56    안녕 하 세요 저 는 그냥 평범 대학 생 며 고등학생 인 자폐 급 남동생 을 둔 누...
Name: content_preprocessing, dtype: object

In [129]:
sentences = data['content_preprocessing'].reset_index(drop=True)

## TF-IDF 행렬 만들기

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features= 1000) 
X = vectorizer.fit_transform(sentences)
X.shape

(55486, 1000)

## 토픽 모델링 with sklearn

In [131]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model=LatentDirichletAllocation(n_components=2,learning_method='online',random_state=777,max_iter=1)

In [132]:
lda_top=lda_model.fit_transform(X)

In [133]:
lda_top.shape

(55486, 2)

In [134]:
lda_model.components_.shape

(2, 1000)

In [136]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장

def getTopics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

getTopics(lda_model.components_, terms, 15)

Topic 1: [('세요', 1541.67), ('사람', 1218.84), ('나라', 1087.49), ('아이', 948.56), ('국민', 943.68), ('제발', 806.75), ('입니다', 760.47), ('처벌', 755.23), ('우리', 747.03), ('으로', 743.51), ('생각', 724.73), ('사건', 686.93), ('경찰', 652.73), ('에서', 635.68), ('너무', 623.45)]
Topic 2: [('연금', 2494.46), ('으로', 2175.54), ('에서', 2049.7), ('국민', 2036.93), ('미세먼지', 1605.86), ('입니다', 1344.72), ('병원', 1287.15), ('정부', 1049.87), ('는데', 1008.6), ('장애', 969.67), ('중국', 951.75), ('건강', 891.89), ('보험', 868.37), ('지만', 816.88), ('문제', 811.31)]


- '안전/환경' : 사람, 아이, 국민, 처벌, 사건, 경찰
- '보건복지' : 연급, 병원, 정부, 장애, 건강, 보험

다소 비슷한 주제를 뽑았지만 어느 정도 잘 나누어서 토픽을 뽑은 것 같다.